## Distortion-Based Analysis of Single Transferable Vote (Code)

This file contains code which I (Vikram Kher) have written during my time researching Single Transferable Vote (STV). The program generates a sample election scenario with c candidates and 2^(c-1) voter profiles. It then constructs and solves a linear program (via the library pulp) to find the distortion of STV in the given election scenario.

### Import Necessary Libraries

Note: pulp is a python LP solver

In [1]:
from pulp import *
import math
from string import ascii_uppercase

In [2]:
# Set up LP maximization problem
model = pulp.LpProblem("Distortion_Maximization_problem", LpMaximize)

### Generate Sample Voter Preference Lists

Note: There are many possible election scenarios which are consistent with the rules of STV; this just represents one scenario which I investigated.

In [3]:
#Choose number of candidates in election
c = 4

In [4]:
alpha = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
voters = []
candidates = []

v = 2**(c-1)

# Create list of voter profiles
for i in range(0, v):
    voters.append(i)


# Create list of candidates
for i in range(0,c): 
    candidates.append(alpha[i])
    
preferenceList = dict()
val = []
for i in range(0,v):
    preferenceList[voters[i]] = []
    val.append(0)
    
    
val[0] = 1/2
val[1] = 1/2
def build(preferenceList, i,j):
    for k in range(i,v):
        if(k >= 2**j):
            j = j + 1
        preferenceList[voters[k]].append(candidates[j])
        for q in range(0,c-j-1):
            preferenceList[voters[k]].append(candidates[c-q-1])
    
    build(preferenceList,i+1,j+1)
        
def rebuild(preferenceList, n):
    k = 4
    p = 2
    preferenceList[0].append(candidates[0])
    preferenceList[0].append(candidates[1])
    preferenceList[1].append(candidates[1])
    preferenceList[1].append(candidates[0])
    divide(preferenceList, val, k, p)
                

def divide(preferenceList,val, k, p):
    if k < 2**c-1:
        for i in range(0,int(k/2)):
            #Duplicate preferences
            preferenceList[i].insert(1,candidates[p])
            preferenceList[k/2+i] = preferenceList[i].copy()
            
            #Swap Candidates
            temp = preferenceList[k/2+i][0]
            preferenceList[k/2+i][0] = preferenceList[k/2+i][1]
            preferenceList[k/2+i][1] = temp
            
            #Split frequencies
            val[int(k/2)+i] = val[i]/(p+1)
            val[i]= val[i] - val[int(k/2)+i]
        
        divide(preferenceList, val, k*2,p+1)

rebuild(preferenceList, c)

In [5]:
#Prints voter profiles
print(preferenceList)

{0: ['A', 'D', 'C', 'B'], 1: ['B', 'D', 'C', 'A'], 2: ['C', 'D', 'A', 'B'], 3: ['C', 'D', 'B', 'A'], 4: ['D', 'A', 'C', 'B'], 5: ['D', 'B', 'C', 'A'], 6: ['D', 'C', 'A', 'B'], 7: ['D', 'C', 'B', 'A']}


In [6]:
#Prints frequency of voter profiles amoung voter population
print(val)

[0.25, 0.25, 0.125, 0.125, 0.08333333333333334, 0.08333333333333334, 0.041666666666666664, 0.041666666666666664]


In [7]:
# Prints labels of candidates in election
candidates

['A', 'B', 'C', 'D']

In [8]:
# Prints labels of voters in election
voters

[0, 1, 2, 3, 4, 5, 6, 7]

### Add Variables to LP

In [9]:
voter_candidate_pairs = pulp.LpVariable.dicts("pair",
                                     ((i,j) for i in voters for j in candidates),
                                     lowBound=0,
                                     cat='Continuous')

In [10]:
voter_candidate_pairs

{(0, 'A'): pair_(0,_'A'),
 (0, 'B'): pair_(0,_'B'),
 (0, 'C'): pair_(0,_'C'),
 (0, 'D'): pair_(0,_'D'),
 (1, 'A'): pair_(1,_'A'),
 (1, 'B'): pair_(1,_'B'),
 (1, 'C'): pair_(1,_'C'),
 (1, 'D'): pair_(1,_'D'),
 (2, 'A'): pair_(2,_'A'),
 (2, 'B'): pair_(2,_'B'),
 (2, 'C'): pair_(2,_'C'),
 (2, 'D'): pair_(2,_'D'),
 (3, 'A'): pair_(3,_'A'),
 (3, 'B'): pair_(3,_'B'),
 (3, 'C'): pair_(3,_'C'),
 (3, 'D'): pair_(3,_'D'),
 (4, 'A'): pair_(4,_'A'),
 (4, 'B'): pair_(4,_'B'),
 (4, 'C'): pair_(4,_'C'),
 (4, 'D'): pair_(4,_'D'),
 (5, 'A'): pair_(5,_'A'),
 (5, 'B'): pair_(5,_'B'),
 (5, 'C'): pair_(5,_'C'),
 (5, 'D'): pair_(5,_'D'),
 (6, 'A'): pair_(6,_'A'),
 (6, 'B'): pair_(6,_'B'),
 (6, 'C'): pair_(6,_'C'),
 (6, 'D'): pair_(6,_'D'),
 (7, 'A'): pair_(7,_'A'),
 (7, 'B'): pair_(7,_'B'),
 (7, 'C'): pair_(7,_'C'),
 (7, 'D'): pair_(7,_'D')}

### Add objective function

In [11]:
model += (
    pulp.lpSum([voter_candidate_pairs[(j, candidates[0])]*val[i] for i,j in enumerate(voters)])
)
model

Distortion_Maximization_problem:
MAXIMIZE
0.25*pair_(0,_'A') + 0.25*pair_(1,_'A') + 0.125*pair_(2,_'A') + 0.125*pair_(3,_'A') + 0.08333333333333334*pair_(4,_'A') + 0.08333333333333334*pair_(5,_'A') + 0.041666666666666664*pair_(6,_'A') + 0.041666666666666664*pair_(7,_'A') + 0.0
VARIABLES
pair_(0,_'A') Continuous
pair_(1,_'A') Continuous
pair_(2,_'A') Continuous
pair_(3,_'A') Continuous
pair_(4,_'A') Continuous
pair_(5,_'A') Continuous
pair_(6,_'A') Continuous
pair_(7,_'A') Continuous

### Add Constraints

#### Triangle Inequality

In [12]:
for i in voters:
    for j in candidates:
        for k in voters:
            for l in candidates:
                model += voter_candidate_pairs[(i,j)] <= voter_candidate_pairs[(k,j)] + voter_candidate_pairs[(k,l)] + voter_candidate_pairs[(i,l)]

#### Consistency

In [13]:
for i in voters:
    for j in candidates:
        for k in candidates:
            if(preferenceList[i].index(j) < preferenceList[i].index(k)):
                model += voter_candidate_pairs[(i,j)] <= voter_candidate_pairs[(i,k)]

#### Normalization

In [14]:
model += pulp.lpSum([voter_candidate_pairs[i, candidates[-1]]*val[voters.index(i)] for i in voters]) == 1

#### Optimality

In [15]:
for i in candidates:
    model += pulp.lpSum([voter_candidate_pairs[j, i]*val[voters.index(j)] for j in voters]) >= 1

#### Positive Distances

Already Done

### Solve Model

In [16]:
model.solve()

1

In [17]:
# Print Distortion
print(pulp.value(model.objective))

3.222222225


In [18]:
print("Status:", LpStatus[model.status])

Status: Optimal


In [19]:
for v in model.variables():
    if(v.name[10] == 'A' or v.name[11] == 'A'):
        print(v.name, "=", v.varValue)

pair_(0,_'A') = 1.7777778
pair_(1,_'A') = 4.4444444
pair_(2,_'A') = 2.6666667
pair_(3,_'A') = 4.4444444
pair_(4,_'A') = 2.6666667
pair_(5,_'A') = 3.5555556
pair_(6,_'A') = 2.6666667
pair_(7,_'A') = 3.5555556


### Extract dual variable values

In [20]:
for name, c in list(model.constraints.items()):
    # Print non-negative dual variables
    if(c.pi != 0):
        print(name, ":", c, "\t Dual", c.pi)

_C132 : -pair_(0,_'A') - pair_(0,_'D') + pair_(1,_'A') - pair_(1,_'D') <= 0 	 Dual 0.11574074
_C148 : pair_(1,_'A') - pair_(1,_'D') - pair_(4,_'A') - pair_(4,_'D') <= 0 	 Dual 0.087962963
_C156 : pair_(1,_'A') - pair_(1,_'D') - pair_(6,_'A') - pair_(6,_'D') <= 0 	 Dual 0.046296296
_C296 : -pair_(1,_'B') - pair_(1,_'D') + pair_(2,_'B') - pair_(2,_'D') <= 0 	 Dual 0.2037037
_C312 : pair_(2,_'B') - pair_(2,_'D') - pair_(5,_'B') - pair_(5,_'D') <= 0 	 Dual 0.0046296296
_C388 : -pair_(0,_'A') - pair_(0,_'D') + pair_(3,_'A') - pair_(3,_'D') <= 0 	 Dual 0.078703704
_C416 : pair_(3,_'A') - pair_(3,_'D') - pair_(7,_'A') - pair_(7,_'D') <= 0 	 Dual 0.046296296
_C552 : -pair_(1,_'B') - pair_(1,_'D') + pair_(4,_'B') - pair_(4,_'D') <= 0 	 Dual 0.074074074
_C588 : -pair_(2,_'C') - pair_(2,_'D') + pair_(4,_'C') - pair_(4,_'D') <= 0 	 Dual 0.055555556
_C592 : -pair_(3,_'C') - pair_(3,_'D') + pair_(4,_'C') - pair_(4,_'D') <= 0 	 Dual 0.046296296
_C644 : -pair_(0,_'A') - pair_(0,_'D') + pair_(5,_'A') -